# Análise exploratória e testes amostral - linha 663

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória

Preparar o ambiente:

In [53]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [54]:
servico = "663"

#### 1 - Importar e tratar os dados da amostra

In [55]:
amostra = pd.read_excel('../data/raw/Relatorio de Viagens das linhas 605_LECD50. 663 e 844.xlsx')
amostra_663 =  amostra[amostra['Linha'] == 663]

# padronizar as colunas da amostra 
amostra_663 = amostra_663.rename(
    columns={"Data": "data",
             "Hora_Inicio": "hora_inicio",
             "Linha": "servico",
             "Veiculo": "id_veiculo",
             "Hora_Fim": "hora_fim",
             "Sentido": "sentido"
             }
    )

# setar tipos de dados e criar colunas com o datetime
amostra_663['data'] = amostra_663['data'].astype(str)
amostra_663['hora_inicio'] = amostra_663['hora_inicio'].astype(str)
amostra_663['datetime_partida'] = pd.to_datetime(amostra_663['data'] + ' ' + amostra_663['hora_inicio'])
amostra_663['hora_fim'] = amostra_663['hora_fim'].astype(str)
amostra_663['datetime_chegada'] = pd.to_datetime(amostra_663['data'] + ' ' + amostra_663['hora_fim'])

amostra_663

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00
5,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00
6,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00
7,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00
8,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00
9,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00


#### 2 - Classificar viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [56]:
amostra_663 = remove_overlapping_trips(amostra_663)
amostra_663

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,NaN
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN
5,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN
6,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN
7,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN
8,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,NaN
9,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN


### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [57]:
# separar as datas da amostra para fazer a query
datas = amostra_663['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_completa_663 = bd.read_sql(q, from_file=True)
viagem_completa_663.info() 
viagem_completa_663      

Downloading: 100%|██████████| 15/15 [00:00<00:00, 30.61rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         15 non-null     object        
 1   servico_informado  15 non-null     object        
 2   sentido            15 non-null     object        
 3   datetime_partida   15 non-null     datetime64[ns]
 4   datetime_chegada   15 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 728.0+ bytes


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,B28514,663,C,2022-09-27 05:51:39,2022-09-27 07:37:09
1,B28600,663,C,2022-09-27 06:15:13,2022-09-27 07:56:13
2,B28570,663,V,2023-07-05 07:05:24,2023-07-05 07:54:24
3,B28567,663,V,2023-07-05 07:22:02,2023-07-05 08:06:32
4,B28570,663,I,2023-07-05 05:57:54,2023-07-05 06:51:54
5,B28567,663,I,2023-07-05 06:22:02,2023-07-05 07:17:32
6,B28567,663,V,2023-07-13 07:36:03,2023-07-13 08:20:03
7,B28570,663,V,2023-07-13 07:06:19,2023-07-13 07:59:49
8,B28567,663,I,2023-07-13 06:23:03,2023-07-13 07:23:33
9,B28570,663,I,2023-07-13 05:58:19,2023-07-13 06:51:49


In [58]:
datas

array(['2022-09-14', '2022-09-22', '2022-09-27', '2023-07-05',
       '2023-07-13', '2023-07-18'], dtype=object)

Esta etapa cruza dados da amostra com as viagens completas para o serviço 663 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [59]:
amostra_663 = check_complete_trips(amostra_663, viagem_completa_663, 10)
amostra_663

Não existem casos duplicados no cruzamento de dados.


,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,Viagem circular identificada e já paga,663,B28514,C,2022-09-14 05:49:03,2022-09-14 07:32:33
1,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,Viagem circular identificada e já paga,663,B28514,C,2022-09-27 05:51:39,2022-09-27 07:37:09
2,2022-09-27,663,B28600,06:14:00,07:07:00,I,2022-09-27 06:14:00,2022-09-27 07:07:00,Viagem circular identificada e já paga,663,B28600,C,2022-09-27 06:15:13,2022-09-27 07:56:13
3,2023-07-05,663,B28570,05:56:00,06:57:00,I,2023-07-05 05:56:00,2023-07-05 06:57:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-05 05:57:54,2023-07-05 06:51:54
4,2023-07-05,663,B28570,07:03:00,07:57:00,V,2023-07-05 07:03:00,2023-07-05 07:57:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-05 07:05:24,2023-07-05 07:54:24
5,2023-07-13,663,B28570,05:57:00,06:52:00,I,2023-07-13 05:57:00,2023-07-13 06:52:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-13 05:58:19,2023-07-13 06:51:49
6,2023-07-13,663,B28570,07:04:00,07:59:00,V,2023-07-13 07:04:00,2023-07-13 07:59:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-13 07:06:19,2023-07-13 07:59:49
7,2023-07-18,663,B28570,05:55:00,06:49:00,I,2023-07-18 05:55:00,2023-07-18 06:49:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-18 05:56:43,2023-07-18 06:48:13
8,2023-07-18,663,B28570,07:02:00,07:54:00,V,2023-07-18 07:02:00,2023-07-18 07:54:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-18 07:04:13,2023-07-18 07:51:13
9,2023-07-05,663,B28567,06:20:00,07:18:00,I,2023-07-05 06:20:00,2023-07-05 07:18:00,Viagem circular identificada e já paga,663,B28567,I,2023-07-05 06:22:02,2023-07-05 07:17:32


Em 13 das 22 viagens válidas da amostra que foram identificadas, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

#### 3.2 Consultar se as viagens da amostra foram feitas por veículos diferentes daqueles indicados na amostra

As viagens não identificadas anteriormente na amostra foram comparadas com as viagens dos veículos indicados na amostra, considerando um intervalo de + - 10 minutos do `datetime_partida`.

In [60]:
amostra_663_sem_status = amostra_663[amostra_663['status'].isna()]
amostra_663_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
15,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN,NaN,NaN,NaN,NaT,NaT
16,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN,NaN,NaN,NaN,NaT,NaT
17,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN,NaN,NaN,NaN,NaT,NaT
18,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN,NaN,NaN,NaN,NaT,NaT
19,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN,NaN,NaN,NaN,NaT,NaT
20,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN,NaN,NaN,NaN,NaT,NaT
21,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN,NaN,NaN,NaN,NaT,NaT
22,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN,NaN,NaN,NaN,NaT,NaT
23,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,NaN,NaN,NaN,NaN,NaT,NaT


In [61]:
id_veiculo = amostra_663_sem_status['id_veiculo_amostra'].unique()
datas = amostra_663_sem_status['data'].unique()

q = f"""
       SELECT
         data, id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)} AND
         id_veiculo IN {tuple(id_veiculo)}
       """
       
viagem_completa_veiculo = bd.read_sql(q, from_file=True)
viagem_completa_veiculo.info()  

Downloading: 100%|██████████| 136/136 [00:00<00:00, 361.70rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               136 non-null    dbdate        
 1   id_veiculo         136 non-null    object        
 2   servico_informado  136 non-null    object        
 3   sentido            136 non-null    object        
 4   datetime_partida   136 non-null    datetime64[ns]
 5   datetime_chegada   136 non-null    datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 6.5+ KB


Nos dias mencionados na amostra, os veículos também operaram nestes serviços:

In [62]:
viagem_completa_veiculo['servico_informado'].unique()

array(['616', '663', '696'], dtype=object)

In [63]:
amostra_663_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
15,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN,NaN,NaN,NaN,NaT,NaT
16,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN,NaN,NaN,NaN,NaT,NaT
17,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN,NaN,NaN,NaN,NaT,NaT
18,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN,NaN,NaN,NaN,NaT,NaT
19,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN,NaN,NaN,NaN,NaT,NaT
20,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN,NaN,NaN,NaN,NaT,NaT
21,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN,NaN,NaN,NaN,NaT,NaT
22,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN,NaN,NaN,NaN,NaT,NaT
23,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,NaN,NaN,NaN,NaN,NaT,NaT


Verificar se os veículos fizeram viagens em outros serviços dentro do intervalo de 10 minutos:

In [64]:
amostra_663_sem_status['servico'] = amostra_663_sem_status['servico'].astype(str)
viagem_completa_veiculo['viagem_completa_veiculo'] = viagem_completa_veiculo['servico_informado'].astype(str)

# 1. Adicionar uma chave temporária
amostra_663_sem_status['tmp_key'] = amostra_663_sem_status['servico']
viagem_completa_veiculo['tmp_key'] = viagem_completa_veiculo['servico_informado']

# 2. Fazendo o merge usando a chave temporária
tabela_comparativa = pd.merge(amostra_663_sem_status, viagem_completa_veiculo, 
                               on='tmp_key')


# # # 3. Filtrar os resultados com base no critério do intervalo de tempo
condition = (tabela_comparativa['datetime_partida_apurada'] >= (tabela_comparativa['datetime_partida_amostra'] - pd.Timedelta(minutes=10))) & \
             (tabela_comparativa['datetime_partida_apurada'] <= (tabela_comparativa['datetime_partida_amostra'] + pd.Timedelta(minutes=10)))

tabela_comparativa = tabela_comparativa[condition]

# # # Removendo a chave temporária e outras colunas desnecessárias
tabela_comparativa.drop(columns=['tmp_key'], inplace=True)

tabela_comparativa

,data_x,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado_x,...,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada,data_y,id_veiculo,servico_informado_y,sentido,datetime_partida,datetime_chegada,viagem_completa_veiculo


Os veículos das viagens não encontradas não fizeram viagens em outros serviços nos horários indicados pela amostra.

#### 3.3 Verificar dados de Conformidade

In [65]:
# separar as datas da amostra para fazer a query
datas = amostra_663['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, 
         datetime_chegada, perc_conformidade_shape, 
         perc_conformidade_registros
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_conformidade = bd.read_sql(q, from_file=True)
viagem_conformidade.info() 
viagem_conformidade    

Downloading: 100%|██████████| 15/15 [00:00<00:00, 46.73rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id_veiculo                   15 non-null     object        
 1   servico_informado            15 non-null     object        
 2   sentido                      15 non-null     object        
 3   datetime_partida             15 non-null     datetime64[ns]
 4   datetime_chegada             15 non-null     datetime64[ns]
 5   perc_conformidade_shape      15 non-null     float64       
 6   perc_conformidade_registros  15 non-null     float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 968.0+ bytes


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_registros
0,B28514,663,C,2022-09-27 05:51:39,2022-09-27 07:37:09,100.0,94.39
1,B28600,663,C,2022-09-27 06:15:13,2022-09-27 07:56:13,100.0,93.14
2,B28514,663,C,2022-09-14 05:49:03,2022-09-14 07:32:33,100.0,95.19
3,B28570,663,V,2023-07-05 07:05:24,2023-07-05 07:54:24,100.0,100.00
4,B28567,663,V,2023-07-05 07:22:02,2023-07-05 08:06:32,100.0,100.00
5,B28570,663,I,2023-07-05 05:57:54,2023-07-05 06:51:54,100.0,100.00
6,B28567,663,I,2023-07-05 06:22:02,2023-07-05 07:17:32,100.0,100.00
7,B28570,663,V,2023-07-18 07:04:13,2023-07-18 07:51:13,100.0,100.00
8,B28567,663,V,2023-07-18 07:24:07,2023-07-18 08:16:37,100.0,100.00
9,B28567,663,I,2023-07-18 06:23:07,2023-07-18 07:13:07,100.0,100.00


In [66]:
amostra_663_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada,tmp_key
15,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN,NaN,NaN,NaN,NaT,NaT,663
16,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN,NaN,NaN,NaN,NaT,NaT,663
17,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN,NaN,NaN,NaN,NaT,NaT,663
18,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN,NaN,NaN,NaN,NaT,NaT,663
19,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN,NaN,NaN,NaN,NaT,NaT,663
20,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN,NaN,NaN,NaN,NaT,NaT,663
21,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN,NaN,NaN,NaN,NaT,NaT,663
22,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN,NaN,NaN,NaN,NaT,NaT,663
23,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,NaN,NaN,NaN,NaN,NaT,NaT,663


In [67]:
# checar se para os dados que ainda não foram identificados: amostra_663_sem_status

def check_conformity_trips(amostra: pd.DataFrame, viagem_completa: pd.DataFrame, intervalo: int) -> pd.DataFrame:
    
    # Filtrando as colunas necessárias
    viagem_completa = viagem_completa[['servico_informado', 'id_veiculo', 'sentido', 'datetime_partida', 'datetime_chegada']]
    
    # Separando linhas com status NaN e não NaN
    amostra_nan = amostra[amostra['status'].isna()]
    amostra_not_nan = amostra[~amostra['status'].isna()]
       
    amostra_nan = amostra_nan[['data','servico','id_veiculo_amostra','hora_inicio','hora_fim','sentido_amostra',
                               'datetime_partida_amostra','datetime_chegada_amostra','status']]
    
    colunas_para_renomear = {
    'id_veiculo_amostra': 'id_veiculo',
    'sentido_amostra': 'sentido',
    'datetime_partida_amostra': 'datetime_partida',
    'datetime_chegada_amostra': 'datetime_chegada'
    }

    # Renomeando as colunas
    amostra_nan.rename(columns=colunas_para_renomear, inplace=True)
    
    
    # 1. Adicionar uma chave temporária
    amostra_nan['tmp_key'] = amostra_nan['id_veiculo']
    viagem_completa['tmp_key'] = viagem_completa['id_veiculo']

    # 2. Fazendo o merge usando a chave temporária
    tabela_comparativa = pd.merge(amostra_nan, viagem_completa, on='tmp_key', suffixes=('_amostra', '_apurada'))

    # 3. Filtrar os resultados com base no critério do intervalo de tempo
    condition = (tabela_comparativa['datetime_partida_apurada'] >= 
                 (tabela_comparativa['datetime_partida_amostra'] - pd.Timedelta(minutes=intervalo))) & \
                (tabela_comparativa['datetime_partida_apurada'] <= 
                 (tabela_comparativa['datetime_partida_amostra'] + pd.Timedelta(minutes=intervalo)))

    tabela_comparativa = tabela_comparativa[condition]


    # Removendo a chave temporária e outras colunas desnecessárias
    tabela_comparativa.drop(columns=['tmp_key'], inplace=True)

    # Atualizar a coluna 'status' baseada nas condições
    tabela_comparativa.loc[tabela_comparativa['id_veiculo_amostra'] == tabela_comparativa['id_veiculo_apurada'],
                           'status'] = 'Viagem inválida - não atendeu a conformidade'
    
    # Verificar se existem dados duplicados no cruzamento de dados
    unique_data = tabela_comparativa[['id_veiculo_apurada', 'datetime_partida_apurada']].drop_duplicates()
    
    if tabela_comparativa.shape[0] == unique_data.shape[0]:
        print("Não existem casos duplicados no cruzamento de dados.")
    else:        
        duplicated_rows = tabela_comparativa[tabela_comparativa.duplicated(['id_veiculo_apurada', 'datetime_partida_apurada'])]        
        matching_rows = pd.merge(tabela_comparativa, duplicated_rows[['id_veiculo_apurada', 'datetime_partida_apurada']], on=['id_veiculo_apurada', 'datetime_partida_apurada'], how='inner')
        print("\nCasos duplicados encontrados no cruzamento de dados:")
        print(matching_rows)   
   
       
    # juntar tudo em uma tabela só
    
    new_column_names = {
        'id_veiculo': 'id_veiculo_amostra',
        'sentido': 'sentido_amostra',
        'datetime_partida': 'datetime_partida_amostra',
        'datetime_chegada': 'datetime_chegada_amostra'
    }
    
    amostra_not_nan = amostra_not_nan.rename(columns=new_column_names) 
        
    final_data = pd.concat([tabela_comparativa, amostra_not_nan], axis=0).reset_index(drop=True)
    
    
    # Adicionar dados que ainda têm o status NaN
    mask_datetime = amostra['datetime_partida_amostra'].isin(final_data['datetime_partida_amostra'])
    mask_veiculo = amostra['id_veiculo_amostra'].isin(final_data['id_veiculo_amostra'])

    # # colocar nomes corretos das colunas
    tabela_sem_status = amostra[~(mask_datetime & mask_veiculo)]
    tabela_sem_status	
     
    # # join
    tabela_sem_status = tabela_sem_status.rename(columns=new_column_names)      
    final_data = pd.concat([final_data, tabela_sem_status], axis=0).reset_index(drop=True)
    
    return final_data

amostra_663_sem_status = check_conformity_trips(amostra_663_sem_status, viagem_conformidade, 10)

amostra_663_sem_status

Não existem casos duplicados no cruzamento de dados.


,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada,tmp_key
0,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN,NaN,NaN,NaN,NaT,NaT,663
1,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN,NaN,NaN,NaN,NaT,NaT,663
2,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN,NaN,NaN,NaN,NaT,NaT,663
3,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN,NaN,NaN,NaN,NaT,NaT,663
4,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN,NaN,NaN,NaN,NaT,NaT,663
5,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN,NaN,NaN,NaN,NaT,NaT,663
6,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN,NaN,NaN,NaN,NaT,NaT,663
7,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN,NaN,NaN,NaN,NaT,NaT,663
8,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,NaN,NaN,NaN,NaN,NaT,NaT,663


Na amostra, não foi identificada nenhuma viagem que estivesse presente em na tabela `viagem_conformidade` e não estivesse presente em `viagem_completa`, considerando uma margem de 10 minutos para mais ou para menos entre os datetimes de partida. 

#### 3.4 Verificar dados de GPS

In [68]:
data_n_encontradas = amostra_663_sem_status['data'].unique()
data_n_encontradas

array(['2022-09-14', '2022-09-22', '2022-09-27'], dtype=object)

In [69]:
ids_n_encontrados = amostra_663_sem_status['id_veiculo_amostra'].unique()
ids_n_encontrados

array(['B28514', 'B28631', 'B28605', 'B28600'], dtype=object)

In [70]:
data_n_encontradas_gps = amostra_663_sem_status['data'].unique()
ids_n_encontrados_gps = amostra_663_sem_status['id_veiculo_amostra'].unique()

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA IN {tuple(data_n_encontradas_gps)}
  AND id_veiculo IN {tuple(ids_n_encontrados_gps)}
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()

Downloading: 100%|██████████| 23641/23641 [00:05<00:00, 4547.21rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23641 entries, 0 to 23640
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           23641 non-null  object        
 1   servico              23641 non-null  object        
 2   timestamp_gps        23641 non-null  datetime64[ns]
 3   posicao_veiculo_geo  23641 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 738.9+ KB


In [71]:
def set_status(row, df_check):
    # Filter the df_final_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if (filtered_df.iloc[0]['servico'] == row['servico']) and (pd.isna(row['status'])):
            return "Encontrado sinal de GPS para o veículo no mesmo horário e serviço"
    return row['status']

amostra_663_sem_status['status'] = amostra_663_sem_status.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_663_sem_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada,tmp_key
0,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
1,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
2,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
3,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
4,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
5,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
6,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
7,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663
8,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,Encontrado sinal de GPS para o veículo no mesm...,NaN,NaN,NaN,NaT,NaT,663


Apesar de três viagens da amostra não terem sido encontradas na comparação com as viagens completas utilizando o critério de + ou - 10 minutos do horário de partida, elas tiveram horários sobrepostos com viagens que já foram pagas. Este é o caso das três viagens abaixo: 

In [72]:
# juntar todas as 24 linhas e classificar abaixo:
df_final = amostra_663.dropna(subset=['status'])
df_final = pd.concat([df_final, amostra_663_sem_status], ignore_index=True)
df_final = df_final.drop(columns=['tmp_key'])

In [73]:
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28514') & (df_final['datetime_partida_amostra'] == "2022-09-14 06:46:00"), 'status'] = 'Viagem circular identificada e já paga' 
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28514') & (df_final['datetime_partida_amostra'] == "2022-09-27 06:51:00"), 'status'] = 'Viagem circular identificada e já paga' 
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28600') & (df_final['datetime_partida_amostra'] == "2022-09-27 07:10:00"), 'status'] = 'Viagem circular identificada e já paga' 

Para os seis casos de viagens que apresentaram sinal de GPS (`gps_sppo`) no horário indicado na amostra e não foram enquadrados nas classificações anteriores, foram realizadas verificações individuais com o objetivo de comparar os sinais de GPS com o percurso planejado. 

A análise concluiu que nas oito viagens não foram registrados sinais de GPS dentro do raio de 500m nos pontos inicial/final das viagens circulares.

Os mapas gerados a partir desta análise estão disponíveis em `data/figures/mapa_viagens`.

In [74]:
viagem_invalida = 'Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m'

df_final.loc[(df_final['id_veiculo_amostra'] == 'B28514') & (df_final['datetime_partida_amostra'] == "2022-09-22 05:46:00"), 'status'] = viagem_invalida
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28631') & (df_final['datetime_partida_amostra'] == "2022-09-22 06:14:00"), 'status'] = viagem_invalida
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28631') & (df_final['datetime_partida_amostra'] == "2022-09-14 06:16:00"), 'status'] = viagem_invalida
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28605') & (df_final['datetime_partida_amostra'] == "2022-09-14 07:33:00"), 'status'] = viagem_invalida
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28514') & (df_final['datetime_partida_amostra'] == "2022-09-22 06:55:00"), 'status'] = viagem_invalida
df_final.loc[(df_final['id_veiculo_amostra'] == 'B28631') & (df_final['datetime_partida_amostra'] == "2022-09-22 07:07:00"), 'status'] = viagem_invalida

In [75]:
df_final

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,Viagem circular identificada e já paga,663,B28514,C,2022-09-14 05:49:03,2022-09-14 07:32:33
1,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,Viagem circular identificada e já paga,663,B28514,C,2022-09-27 05:51:39,2022-09-27 07:37:09
2,2022-09-27,663,B28600,06:14:00,07:07:00,I,2022-09-27 06:14:00,2022-09-27 07:07:00,Viagem circular identificada e já paga,663,B28600,C,2022-09-27 06:15:13,2022-09-27 07:56:13
3,2023-07-05,663,B28570,05:56:00,06:57:00,I,2023-07-05 05:56:00,2023-07-05 06:57:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-05 05:57:54,2023-07-05 06:51:54
4,2023-07-05,663,B28570,07:03:00,07:57:00,V,2023-07-05 07:03:00,2023-07-05 07:57:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-05 07:05:24,2023-07-05 07:54:24
5,2023-07-13,663,B28570,05:57:00,06:52:00,I,2023-07-13 05:57:00,2023-07-13 06:52:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-13 05:58:19,2023-07-13 06:51:49
6,2023-07-13,663,B28570,07:04:00,07:59:00,V,2023-07-13 07:04:00,2023-07-13 07:59:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-13 07:06:19,2023-07-13 07:59:49
7,2023-07-18,663,B28570,05:55:00,06:49:00,I,2023-07-18 05:55:00,2023-07-18 06:49:00,Viagem circular identificada e já paga,663,B28570,I,2023-07-18 05:56:43,2023-07-18 06:48:13
8,2023-07-18,663,B28570,07:02:00,07:54:00,V,2023-07-18 07:02:00,2023-07-18 07:54:00,Viagem circular identificada e já paga,663,B28570,V,2023-07-18 07:04:13,2023-07-18 07:51:13
9,2023-07-05,663,B28567,06:20:00,07:18:00,I,2023-07-05 06:20:00,2023-07-05 07:18:00,Viagem circular identificada e já paga,663,B28567,I,2023-07-05 06:22:02,2023-07-05 07:17:32


In [76]:
# exportar em csv
df_final.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx', index=False)

# Exemplos (mapas)

Viagem identificada


Veículo: B28514
partida: 2022-09-27 05:51:39	
chegada: 2022-09-27 07:37:09	

amostra: 05:50:00	06:46:00	I	2022-09-27 05:50:00

In [77]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-27"
  AND servico = '663'
"""
      
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.33rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [78]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-09-27"
  AND servico_informado = '663'
  AND timestamp_gps BETWEEN "2022-09-27T05:51:39"
  AND "2022-09-27T07:37:09"
  AND id_veiculo = "B28514"
"""

gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 198/198 [00:00<00:00, 282.05rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           198 non-null    object        
 1   servico              198 non-null    object        
 2   timestamp_gps        198 non-null    datetime64[ns]
 3   posicao_veiculo_geo  198 non-null    object        
 4   status_viagem        198 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 7.9+ KB


In [79]:
create_trip_map(gps_identificado, shape_identificado)

Viagem não identificada

B28631	14-09-2022 entre 06:16:00 e 08:13:00


In [89]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-14"
  AND servico = '663'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.35rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [90]:
q = f"""
SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-14"
  AND timestamp_gps BETWEEN "2022-09-14T06:16:00"
  AND "2022-09-14T08:13:00"
  AND id_veiculo = "B28631"
"""

gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 100%|██████████| 214/214 [00:00<00:00, 620.29rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           214 non-null    object        
 1   servico              214 non-null    object        
 2   timestamp_gps        214 non-null    datetime64[ns]
 3   posicao_veiculo_geo  214 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 6.8+ KB


In [91]:
create_trip_map(gps_n_identificado, shape_n_identificado)

Viagem não identificada 2:

B28514 22-09-2022 05:46:00 06:36:00

In [83]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-22"
  AND servico = '663'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.46rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [84]:
q = f"""
SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-22"
  AND timestamp_gps BETWEEN "2022-09-22T05:46:00"
  AND "2022-09-22T07:45:00"
  AND id_veiculo = "B28514"
"""

gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 100%|██████████| 217/217 [00:00<00:00, 549.37rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           217 non-null    object        
 1   servico              217 non-null    object        
 2   timestamp_gps        217 non-null    datetime64[ns]
 3   posicao_veiculo_geo  217 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 6.9+ KB


In [85]:
create_trip_map(gps_n_identificado, shape_n_identificado)

## (2) Gráficos da análise exploratória / análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [86]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '663'
  AND DATA BETWEEN '2022-06-01' AND '2023-09-14'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 390/390 [00:01<00:00, 250.48rows/s]


In [87]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [88]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()